# Previamente creamos el kernel para este proyecto
uv run python -m ipykernel install \
  --user \
  --name polarsnba \
  --display-name "Python (polarsnba)"

In [1]:
import polars as pl

# DATASET SACADO DE KAGGLE (https://www.kaggle.com/datasets/eoinamoore/historical-nba-data-and-player-box-scores)

## LIMPIEZA PLAYERS

In [2]:
df_players = pl.read_csv('/home/uvuser/polarsNBA/archive/raw/Players.csv')

In [3]:
df_players

personId,firstName,lastName,birthdate,lastAttended,country,height,bodyWeight,guard,forward,center,draftYear,draftRound,draftNumber
i64,str,str,str,str,str,f64,f64,bool,bool,bool,f64,f64,f64
2,"""Byron""","""Scott""","""1961-03-28""","""Arizona State""","""USA""",76.0,205.0,true,false,false,1983.0,1.0,4.0
3,"""Grant""","""Long""","""1966-03-12""","""Eastern Michigan""","""USA""",81.0,3.0,false,false,false,1988.0,2.0,33.0
7,"""Dan""","""Schayes""","""1959-05-10""","""Syracuse""","""USA""",83.0,260.0,false,false,true,1981.0,1.0,13.0
9,"""Sedale""","""Threatt""","""1961-09-10""","""West Virginia Tech""","""USA""",74.0,185.0,true,false,false,1983.0,6.0,139.0
12,"""Chris""","""King""","""1969-07-24""","""Wake Forest""","""USA""",80.0,215.0,false,true,false,1992.0,2.0,45.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
1966938196,"""Yuheng""","""Tian""",null,null,null,null,null,null,null,null,null,null,null
1966938197,"""Tianyo""","""Yao""",null,null,null,null,null,null,null,null,null,null,null
1966938204,"""Koperberg""","""Daniel""",null,null,null,null,null,null,null,null,null,null,null


In [4]:
# Contar los nulos por columna
nulos_por_columna = df_players.select([
    pl.col(col).is_null().sum().alias(f"{col}_nulos")
    for col in df_players.columns
])

print(nulos_por_columna)

shape: (1, 14)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ personId_ ┆ firstName ┆ lastName_ ┆ birthdate ┆ … ┆ center_nu ┆ draftYear ┆ draftRoun ┆ draftNum │
│ nulos     ┆ _nulos    ┆ nulos     ┆ _nulos    ┆   ┆ los       ┆ _nulos    ┆ d_nulos   ┆ ber_nulo │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ s        │
│ u32       ┆ u32       ┆ u32       ┆ u32       ┆   ┆ u32       ┆ u32       ┆ u32       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ u32      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 0         ┆ 1         ┆ 2         ┆ 1856      ┆ … ┆ 1430      ┆ 1856      ┆ 2017      ┆ 2075     │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘


In [5]:
duplicados = df_players.filter(df_players.is_duplicated())
print(duplicados)

shape: (0, 14)
┌──────────┬───────────┬──────────┬───────────┬───┬────────┬───────────┬────────────┬─────────────┐
│ personId ┆ firstName ┆ lastName ┆ birthdate ┆ … ┆ center ┆ draftYear ┆ draftRound ┆ draftNumber │
│ ---      ┆ ---       ┆ ---      ┆ ---       ┆   ┆ ---    ┆ ---       ┆ ---        ┆ ---         │
│ i64      ┆ str       ┆ str      ┆ str       ┆   ┆ bool   ┆ f64       ┆ f64        ┆ f64         │
╞══════════╪═══════════╪══════════╪═══════════╪═══╪════════╪═══════════╪════════════╪═════════════╡
└──────────┴───────────┴──────────┴───────────┴───┴────────┴───────────┴────────────┴─────────────┘


In [6]:
num_duplicados = df_players.is_duplicated().sum()
print(f"Número de filas duplicadas: {num_duplicados}")

Número de filas duplicadas: 0


In [7]:
df_players.describe()

statistic,personId,firstName,lastName,birthdate,lastAttended,country,height,bodyWeight,guard,forward,center,draftYear,draftRound,draftNumber
str,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",6533.0,"""6532""","""6531""","""4677""","""4654""","""4677""",4454.0,4580.0,5103.0,5103.0,5103.0,4677.0,4516.0,4458.0
"""null_count""",0.0,"""1""","""2""","""1856""","""1879""","""1856""",2079.0,1953.0,1430.0,1430.0,1430.0,1856.0,2017.0,2075.0
"""mean""",4.3522e7,null,null,null,null,null,78.031881,102650.384716,0.388595,0.415638,0.16794,1397.4257,-0.542073,20.775236
"""std""",2.7516e8,null,null,null,null,null,3.629342,376037.813095,null,null,null,910.019928,6.71421,31.320861
"""min""",2.0,"""A.C.""","""Abalde""","""1900-01-01""","""--""","""Anla""",65.0,3.0,0.0,0.0,0.0,-22.0,-22.0,-22.0
"""25%""",56046.0,null,null,null,null,null,75.0,195.0,null,null,null,-1.0,-1.0,-1.0
"""50%""",77839.0,null,null,null,null,null,78.0,216.0,null,null,null,1975.0,1.0,14.0
"""75%""",203965.0,null,null,null,null,null,81.0,250.0,null,null,null,1996.0,2.0,36.0
"""max""",1.9669e9,"""ga""","""de la Fuente""","""2005-12-30""","""nzaga""","""Venezuela""",91.0,1.64253e6,1.0,1.0,1.0,2024.0,20.0,221.0
